# Training

In [1]:
%load_ext tensorboard

In [1]:
import sys

In [2]:
sys.path.append("/home/idumeur/code")
sys.path.append("/home/idumeur/code/sent2_cloud_remover/")

In [3]:
from utils.load_dataset import load_data
from models import clean_gan
from train import open_yaml,saving_yaml

AttributeError: module 'tensorflow' has no attribute 'compat'

In [ ]:
PATH="/srv/osirim/idumeur/data/dataset6/prepro1/input_large_dataset/" #path to the dataset which contains the tile for the training
PATH_TRAININGS="/srv/osirim/idumeur/trainings/"
NAME_MODEL="Model0"
TRAINING_NBER="1"
PATH_THIS_MODEL=PATH_TRAININGS+NAME_MODEL+"/"
PATH_THIS_TRAINING="{}{}/training_{}/".format(PATH_TRAININGS,NAME_MODEL,TRAINING_NBER)
PATH_CHECKPOINT="{}checkpoints/".format(PATH_THIS_TRAINING)
PATH_SAVED_IM="{}saved_training_images/".format(PATH_THIS_TRAINING)
PATH_TRAIN_YAML="/home/idumeur/code/sent2_cloud_remover/GAN_confs/train.yaml"
PATH_MODEL_YAML="/home/idumeur/code/sent2_cloud_remover/GAN_confs/model.yaml"

In [6]:
train_param=open_yaml(PATH_TRAIN_YAML)
model_param=open_yaml(PATH_MODEL_YAML)

/home/idumeur/code/sent2_cloud_remover/train.py:43: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  return yaml.load(f)


In [7]:
model_param["dict_discri_archi"]

{1: [1, 2, 4, 64],
 2: [1, 2, 4, 256],
 3: [1, 2, 4, 156],
 4: [1, 1, 4, 512],
 5: [1, 1, 4, 1]}

In [8]:
train_param["train_directory"]=PATH+"train/"
train_param["val_directory"]=PATH+"val/"
train_param["training_number"]=1

In [9]:
train_param["normalization"]=True #id set to true normalization to the data is apllied, the norm implemented is standardization for SAR data and normalization for RGBNIR
train_param["dict_band_x"]=None #the stats for normalization procedure is going to be compute on each of these group of bands
train_param["dict_band_label"]=None
train_param["dict_rescale_type"]=None
train_param["training_dir"]="/srv/osirim/idumeur/trainings/"


In [12]:
gan=clean_gan.GAN(model_param, train_param)

No scaler was defined before
No scaler was defined before
No scaler was defined before
Loading the data done dataX (503, 256, 256, 8) dataY 
Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
d_input (InputLayer)         [(None, 256, 256, 12)]    0         
_________________________________________________________________
d_GaussianNoise (GaussianNoi (None, 256, 256, 12)      0         
_________________________________________________________________
d_pad_1 (ZeroPadding2D)      (None, 258, 258, 12)      0         
_________________________________________________________________
d_conv1 (Conv2D)             (None, 128, 128, 64)      12352     
_________________________________________________________________
d_pad_2 (ZeroPadding2D)      (None, 130, 130, 64)      0         
_________________________________________________________________
d_conv2 (Conv2D)             (None, 64, 64, 

G Tensor("Generator_1/Identity:0", shape=(None, 256, 256, 4), dtype=float32)
INPUT DISCRI  Tensor("concat_1:0", shape=(None, 256, 256, 12), dtype=float32)
[INFO] combined model loss are : 


In [16]:
model_dir = gan.model_dir
training_dir = gan.this_training_dir
#saving_yaml(path_model, model_dir)
#saving_yaml(path_train, training_dir)
gan.train()

starting epoch 0
0 iter 0 [D loss: 0.776601, acc.: 51.17%] [G loss: 0.754766 0.754766]


/home/idumeur/code/sent2_cloud_remover/utils/metrics.py:56: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  lssim_batch += [ssim(batch1[i, :, :, :], batch2[i, :, :, :], multichannel=True)]
/home/idumeur/miniconda3/envs/tf_env/lib/python3.6/site-packages/skimage/metrics/_structural_similarity.py:108: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  im2[..., ch], **args)


0 iter 1 [D loss: 0.783189, acc.: 53.89%] [G loss: 0.775377 0.775377]
0 iter 2 [D loss: 0.776314, acc.: 50.17%] [G loss: 0.763707 0.763707]
0 iter 3 [D loss: 0.770924, acc.: 52.39%] [G loss: 0.751824 0.751824]
0 iter 4 [D loss: 0.764195, acc.: 52.11%] [G loss: 0.748073 0.748073]
0 iter 5 [D loss: 0.778446, acc.: 52.06%] [G loss: 0.766931 0.766931]
0 iter 6 [D loss: 0.744789, acc.: 49.78%] [G loss: 0.735323 0.735323]
0 iter 7 [D loss: 0.771588, acc.: 49.83%] [G loss: 0.765574 0.765574]
0 iter 8 [D loss: 0.766596, acc.: 51.50%] [G loss: 0.758286 0.758286]
0 iter 9 [D loss: 0.757229, acc.: 48.39%] [G loss: 0.753821 0.753821]
0 iter 10 [D loss: 0.761808, acc.: 48.72%] [G loss: 0.758823 0.758823]
0 iter 11 [D loss: 0.735083, acc.: 53.28%] [G loss: 0.731068 0.731068]
0 iter 12 [D loss: 0.757916, acc.: 49.06%] [G loss: 0.751145 0.751145]
0 iter 13 [D loss: 0.726275, acc.: 52.00%] [G loss: 0.721702 0.721702]
0 iter 14 [D loss: 0.715848, acc.: 50.39%] [G loss: 0.718992 0.718992]
0 iter 15 [D lo

KeyboardInterrupt: 